In [ ]:
import tensorflow as tf
import optuna
# from sklearn.model_selection import train_test_split

# Sample data (replace with your actual data)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 28 * 28).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28 * 28).astype("float32") / 255.0

def create_model(learning_rate):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(784,)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def objective(trial):
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    model = create_model(learning_rate)
    model.fit(x_train, y_train, epochs=3, validation_split=0.2, verbose=0)
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10) # Adjust n_trials as needed

print('Number of finished trials: ', len(study.trials))
print('Best trial:')
trial = study.best_trial
print('  Value: ', trial.value)
print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

best_learning_rate = study.best_params['learning_rate']

# # Retrain with the best learning rate.
# best_model = create_model(best_learning_rate)
# best_model.fit(x_train, y_train, epochs=10, validation_split=0.2)
# final_loss, final_accuracy = best_model.evaluate(x_test,y_test)

# print(f"Final Test Accuracy with best LR: {final_accuracy}")

In [ ]:
|